In [1]:
import sys
sys.path.insert(0, "../")

from sklearn.ensemble import RandomForestClassifier

from src.utils import *
from src.synthesizer import *

from sklearn.utils import shuffle
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, roc_auc_score
from copy import deepcopy

In [2]:
dataset_name = 'drug'
column_metric = "Ethnicity"
 
seed=0
seed_everything(seed)

In [3]:
from copy import deepcopy
import pandas as pd

def uncertainty(column_metric, ctgans, trained_model_dict, random_state=0, n_samples=5000, test_group='small', smallest_group = 3):
        from sklearn.metrics import f1_score

        score='acc'

        n_samples=1000
       
        groups = list(np.unique(Data[column_metric]))
        orig_random_state = deepcopy(random_state)


        for model in ['rf']:#trained_model_dict.keys():


            random_state = orig_random_state


            try:
                trained_model_dict[model].predict_proba(X_train.drop('y', axis=1))
            except:
                continue

            means_5 = []
            stds_5 = []

            means_10 = []
            stds_10 = []

            oracle_score = []
            real_test_score = []


            for random_state in range(1):


                #smallest_group = 0 
                gsizes = []
                for idx, group in enumerate(groups):
                    gsizes.append(X_val[X_val[column_metric]==group].shape[0])


                ns=5
                for group in tqdm(groups):

                    clf = model_dict[model]


                    if group!=smallest_group:
                        continue
                    
                    #n_samples = 1000
                    if group==0:
                        total_samples = X_val[X_val[column_metric]==0].shape[0]
                    else:
                        total_samples = n_samples 



                    test_data = X_val[X_val[column_metric]==group]

                    oracle_data = X_test[X_test[column_metric]==group]

   

                    y_pred = clf.predict(oracle_data.drop('y', axis=1))

                    accuracy = accuracy_score(oracle_data['y'], y_pred)

                    oracle_score.append(accuracy)

                    try:
                        y_pred = clf.predict(test_data.drop('y', axis=1))
                        accuracy = accuracy_score(test_data['y'], y_pred)
                    except:
                        accuracy=0.5
                    real_test_score.append(accuracy)

                    
                    if test_data.shape[0]==0:
                        accuracy = 0
                        TPR=0
                        FPR=0
                        TNR=0
                        FNR=0
                        F1=0
                        eo_score = 0
                        dp_score = 0
                    
                    else:
                        
                        #K=5
                        syn_accs2 = []
                        for ctgan in ctgans[0:5]:
                            shift_df, _ = ctgan.sample(1, shift=False, condition_column=column_metric, condition_value=group)

                            count=0
                            while shift_df.shape[0]<=total_samples:

                                generated_tmp, _ = ctgan.sample(n_samples, shift=False, condition_column=column_metric, condition_value=group)

                                tmp_df = generated_tmp[generated_tmp[column_metric]==group]

                                shift_df = shift_df.append(tmp_df)
                                count+=1

                            shift_df.shape, count

                            syn_data = shift_df[shift_df[column_metric]==group]

                            y_pred = clf.predict(syn_data.drop('y', axis=1))

                            accuracy = accuracy_score(syn_data['y'], y_pred)

                            syn_accs2.append(accuracy)



                        mean_syn_acc = np.mean(syn_accs2)
                        std_syn_acc = np.std(syn_accs2)


                        means_5.append(mean_syn_acc)
                        stds_5.append(std_syn_acc)


                        # K=10
                        syn_accs3 = []
                        for ctgan in ctgans:
                            shift_df, _ = ctgan.sample(1, shift=False, condition_column=column_metric, condition_value=group)


                            count=0
                            while shift_df.shape[0]<=total_samples:

                                generated_tmp, _ = ctgan.sample(n_samples, shift=False, condition_column=column_metric, condition_value=group)

                                tmp_df = generated_tmp[generated_tmp[column_metric]==group]

                                shift_df = shift_df.append(tmp_df)
                                count+=1

                            shift_df.shape, count

                            syn_data = shift_df[shift_df[column_metric]==group]

                            y_pred = clf.predict(syn_data.drop('y', axis=1))

                            accuracy = accuracy_score(syn_data['y'], y_pred)

                            syn_accs3.append(accuracy)


                        mean_syn_acc = np.mean(syn_accs3)
                        std_syn_acc = np.std(syn_accs3)


                        means_10.append(mean_syn_acc)
                        stds_10.append(std_syn_acc)
   
 
        return means_5, means_10, stds_5, stds_10, oracle_score, real_test_score


In [4]:
from src.data_loader import load_drug_dataset

X, y, Data = load_drug_dataset()

In [5]:
import pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

res_loop = []

discrete_columns = [
      'Age', 'Gender', 'Education', 'Country', column_metric, 'Impulsive', 'SS', 'Alcohol',
       'Amphet', 'Amyl', 'Benzos', 'Cannabis', 'Coke', 'Crack', 'Ecstasy',
       'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'VSA', 'y'
    ]

groups = list(Data[column_metric].unique())


for testsize in tqdm([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]):

    res = {}

    reslist = {}

    for seed in range(1):
        
        seed=100
        seed_everything(seed=seed)
    
        X_train, X_test = train_test_split(Data, test_size=0.3, random_state=seed)

        X_train, X_val = train_test_split(X_train, test_size=0.8, random_state=seed)

        _, X_val = train_test_split(X_val, test_size=testsize, random_state=seed)

        gsizes = []
        for idx, group in enumerate(groups):
            gsizes.append(X_val[X_val[column_metric]==group].shape[0])


        ctgans = []
        trials = []
        params_list = []

        

        from copy import deepcopy
        from tqdm import tqdm
        for i in range(10):

            best_params = {'learning_rate': 0.0002, 'embedding_dim': 256, 'epochs': 100}
            ctgan = fit_ctgan(data=X_val, epochs=best_params['epochs'], learning_rate=best_params['learning_rate'], embedding_dim=best_params['embedding_dim'],seed=seed+i, discrete_columns=discrete_columns)

            # trials.append(trial_results)
            params_list.append(best_params)
            ctgans.append(deepcopy(ctgan))


        # save each ctgan model in ctgan_list
        for idx, ctgan_save in enumerate(ctgans):
            ctgan_save.save(f'../models/ctgan_{dataset_name}_{idx+1}')



        #pickle best params
        with open(f'../models/ctgan_{dataset_name}_params.pkl', 'wb') as f:
            pickle.dump(params_list[0], f)



        model_dict = {
                    'rf': RandomForestClassifier(random_state=0),
                }

        print('training baseline models')
        trained_model_dict = {}

        for model in model_dict.keys():
            clf = model_dict[model]
            clf.fit(X_train.drop('y', axis=1), X_train['y'])

            trained_model_dict[model] = deepcopy(clf)


        column_metric = column_metric
        groups = list(np.unique(Data[column_metric]))


        # load ctgans into a list from file 5 pickles
        ctgans = []
        for i in range(10):
            with open(f'../models/ctgan_{dataset_name}_params.pkl', 'rb') as f:
                best_params = pickle.load(f)

            ctgan = CTGANSynthesizer(embedding_dim=best_params['embedding_dim'], generator_dim=(256, 256), discriminator_dim=(256, 256),
                            generator_lr=best_params['learning_rate'], generator_decay=1e-6, discriminator_lr=best_params['learning_rate'],
                            discriminator_decay=1e-6, batch_size=500, discriminator_steps=1,
                            log_frequency=True, verbose=False, epochs=best_params['epochs'], pac=10, cuda=True)

            ctgan = ctgan.load(f'../models/ctgan_{dataset_name}_{i+1}')
            
            ctgans.append(ctgan)

            from tqdm import tqdm
            
        import warnings
        warnings.simplefilter(action='ignore', category=FutureWarning)


        n_runs=1

        for i in tqdm(range(n_runs)):

            done=False
            tries=0

            while done==False:
                try:
                    means_5, means_10, stds_5, stds_10, oracle_score, test_score = uncertainty(column_metric = column_metric, ctgans=ctgans[0:10], trained_model_dict=trained_model_dict,random_state=i*100, smallest_group=3)
                    done=True
                    #final_val_dataset.append(val_dataset)
                except Exception as e:
                    import traceback
                    print(traceback.format_exc())
                    done=True
                    pass
            
                    
                
        
        try:
            res[testsize] =  {
                            "mean5": means_5,
                            "mean10": means_10,
                            "std5": stds_5,
                            "std10": stds_10,
                            "oracle": oracle_score,
                            "test": test_score,
                            "size": X_val.shape[0],
                            "groupsize": X_val[X_val[column_metric]==3].shape[0]
                        }
        except Exception as e:
            import traceback
            print(traceback.format_exc())
            print(e)
            # pass
    


        try:
            reslist[seed] = res

        except:
            pass


        seed+=1


    res_loop.append(reslist)


  0%|          | 0/7 [00:00<?, ?it/s]

training baseline models



 14%|█▍        | 1/7 [00:53<05:23, 53.84s/it]

training baseline models



 29%|██▊       | 2/7 [01:57<04:58, 59.64s/it]

...


 71%|███████▏  | 5/7 [04:52<01:57, 58.63s/it]

training baseline models



 86%|████████▌ | 6/7 [06:18<01:07, 67.83s/it]

training baseline models



100%|██████████| 7/7 [08:32<00:00, 73.23s/it]


In [13]:
import matplotlib.pyplot as plt

def plot_comparison(dict_list, group=0):
    x = []
    y_mean10_vs_oracle = []
    y_test_vs_oracle = []
    yerr = []

    for data_dict in dict_list:
        for testsize, values in data_dict.items():
            # Skip testsize entries with no samples in the target group
            if values["groupsize"] == 0:
                continue
            if len(values["mean10"]) == 0:
                continue

            # If you want to use a specific group index:
            mean10_value  = values["mean10"][group]
            test_value    = values["test"][group]
            oracle_value  = values["oracle"][group]
            std10_value   = values["std10"][group]

            mean_abs_error_mean10 = abs(oracle_value - mean10_value)
            mean_abs_error_test   = abs(oracle_value - test_value)

            x.append(values["groupsize"])
            y_mean10_vs_oracle.append(mean_abs_error_mean10)
            y_test_vs_oracle.append(mean_abs_error_test)
            yerr.append(std10_value)

    print("Number of points to plot:", len(x))

    plt.errorbar(x, y_mean10_vs_oracle, yerr=yerr, marker="o", linestyle="-", label="3S")
    plt.plot(x, y_test_vs_oracle, marker="o", linestyle="-", label="Dtest")
    plt.xlabel("n Samples in group")
    plt.ylabel("Absolute Error vs Oracle")
    plt.legend()
    plt.show()
